In [27]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
%cd gdrive/MyDrive/Anime_Recommender/Greta_CE

/content/gdrive/MyDrive/Anime_Recommender/Greta_CE


In [ ]:
!pip install sentence_transformers

In [4]:
from sentence_transformers import SentenceTransformer, SentencesDataset, util
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
import cross_encoder_train 
import bi_encoder_train 
import recommender
import os
import csv
import pickle
import time
import sys
import pandas as pd
import torch
from torch.utils.data import DataLoader
import math
import random
from sentence_transformers import SentencesDataset, losses
from sentence_transformers.evaluation import BinaryClassificationEvaluator
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Movie lens data - TODO: Get code to work without truncation. Truncation set to 1000 for now. Additionally, all dataset below except ml-20m//genome-tags.csv are preprocessed to match kaggle data set entries
f1='data/movie_lens_merged_out.csv'
f2='data/ratings_out.csv'
f3='data/genome_score_out.csv'
f4='ml-20m//genome-tags.csv'
f5='data/tags_out.csv'
data=recommender.movie_lens_data(f1,f2,f3,f4,f5, truncate=False, model=1) 

/content/gdrive/MyDrive/Anime_Recommender/Greta_CE/recommender.py:64: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  generes=generes.str.replace('|',' ')  #removing the "|" seperator


In [6]:
data.bert_tokenize()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
# Configure and train Cross-Encoder using Quora Sentence Pairs dataset. Use the filtered 500 synopses as inout to CrossEncoder. A CrossEncoder gets both inputs (input request, synopsis) and outputs similarity score
cross_encoder_model = cross_encoder_train.CE_Model()
cross_encoder_model.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nreimers/MiniLM-L6-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
#Cross-Encoder evaluation post-training
print(f"This is the cross-encoder evaluator result after training: {cross_encoder_model.evaluate()}")

This is the cross-encoder evaluator result after training: 0.8834886997312943


In [9]:
# Configure BiEncoder (SentenceTransformer) that produces embeddings for input request and uses cosine similarity to filter top 500 similar synposes.
bi_encoder_model = bi_encoder_train.BE_Model()


In [10]:
#Data used for recommender (smaller dataset)
dataset_path = "./data/movies_kaggle_merged.csv"

In [11]:
#Check if dataframe exists
df_cache_path = "df_small.pkl"
if os.path.exists(df_cache_path):
    df = pd.read_pickle("./df_small.pkl")  
else:
    df = pd.read_csv(dataset_path)
    df.dropna(subset=['overview'], inplace=True)
    df.drop_duplicates(subset='overview', keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_pickle("./df_small.pkl")  

In [12]:
#Prepare CE output to train Bi-Encoder Model
train_num_samples = 10000 #Can adjust this number
train_sentence_pairs = [] 
df_train = df[:train_num_samples]

for i in range(train_num_samples):
    rand_num = random.randint(0,train_num_samples-1)
    sentence_1=df_train.iloc[rand_num, 4]
    rand_num = random.randint(0,train_num_samples-1)
    sentence_2=df_train.iloc[rand_num, 4]
    train_sentence_pairs.append([sentence_1, sentence_2])

train_ce_scores = cross_encoder_model.model.predict(train_sentence_pairs)
binary_train_ce_scores=[1 if score>.1 else 0 for score in train_ce_scores] #Can tune score threshold




In [24]:
#Train Bi-Encoder using Cross-Encoder output and Quora Sentence Pairs dataset
bi_encoder_model.train(binary_train_ce_scores, train_sentence_pairs)
print(f"This is the bi-encoder evaluator result after training: {bi_encoder_model.evaluate()}")

This is the bi-encoder evaluator result after training: 0.7681989248134373


In [14]:
# File for caching  embeddings
embedding_save_path = 'movies-summary-embeddings-size-small_BE_MNRL.pkl'
embedding_save_path

'movies-summary-embeddings-size-small_BE_MNRL.pkl'

In [19]:
#Check if embedding cache path exists
# Source: https://www.sbert.net/examples/applications/computing-embeddings/README.html
if os.path.exists(embedding_save_path):
    with open(embedding_save_path, "rb") as fIn:
        cached_data = pickle.load(fIn)
        stored_sentences = cached_data['stored_sentences']
        stored_embeddings = cached_data['stored_embeddings']
else:
    stored_sentences = df['overview'].tolist()
    stored_embeddings = bi_encoder_model.model.encode(stored_sentences, show_progress_bar=True, convert_to_tensor=True)   
    with open(embedding_save_path, "wb") as fOut:
        pickle.dump({'stored_sentences': stored_sentences, 'stored_embeddings': stored_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)


In [20]:
corpus_titles = set(df['title'].tolist())

In [ ]:
"""
Sources Used:
https://www.sbert.net/examples/applications/retrieve_rerank/README.html?highlight=rerank
https://www.sbert.net/docs/quickstart.html
Recommend by summary/decription
"""
BE_2_num_movies = 100
while True:
    inp_request = input("Please enter a request: ")

    #First, filter 1400 candidates using bi-encoder 1
    start_time = time.time()
    recommendations = data.get_recommendations_by_keyword_bert_tfidf(inp_request, num_recommendations=1400, bert_weight=.5, tfidf_weight=0.5,
                                                                     w_ratings=False)
    end_time = time.time()
    print(f"First Bi-Encoder search took {(end_time-start_time):.3f} seconds")
    print("Top 5 movies with first Bi-Encoder:")
    for movie in recommendations[:5]:
        print(f"\t{movie}")
    filtered_stored_embedding=[]
    filtered_stored_sentences=[]
    filtered_titles=[]
    titles_seen = set()
    for title in recommendations:
        if title in corpus_titles and title not in titles_seen:
            index = df.index[df['title']==title]
            titles_seen.add(title)
            for idx in index:
                filtered_titles.append(title)
                filtered_stored_sentences.append(stored_sentences[idx])
                filtered_stored_embedding.append(stored_embeddings[idx])
   
    #Second, filter 100 candidates using bi-encoder 2
    start_time = time.time()
    input_embedding = bi_encoder_model.model.encode(inp_request, convert_to_tensor=True)
    
    #Get top 5 results from bi-encoder
    encoder_res = util.semantic_search(input_embedding, filtered_stored_embedding, top_k=BE_2_num_movies)[0]
    end_time = time.time()
    print(f"Second Bi-Encoder search took {(end_time-start_time):.3f} seconds")
    print("Top 5 movies with second Bi-Encoder:")
    for res in encoder_res[:5]:
        print(f"\t{(res['score']):.3f}\t{filtered_titles[res['corpus_id']]}")

    #Use filtered results form bi-encoder as input to cross-encoder
    start_time = time.time()
    sentence_pairs = [[inp_request, filtered_stored_sentences[res['corpus_id']]] for res in encoder_res] #pair all summaries with input request
    ce_scores = cross_encoder_model.model.predict(sentence_pairs)

    encoder_res_len = len(encoder_res)
    for i in range(encoder_res_len):
        encoder_res[i]['cross-encoder_score'] = ce_scores[i]

    #Sort CrossEncoder output results by scores
    encoder_res = sorted(encoder_res, key=lambda x: x['cross-encoder_score'], reverse=True)
    end_time = time.time()
    print(f"\nCross-Encoder Search took {(end_time-start_time):.3f} seconds")
    print("Top 5 movies with Cross-Encoder:")
    for res in encoder_res[:5]:
        print(f"\t{(res['cross-encoder_score']):.3f}\t{filtered_titles[res['corpus_id']]}")


In [25]:
#Recommend by movie title
while True:
    inp_request = input("Please enter a a movie title: ")

    start_time = time.time()
    recommendations = data.get_recommendations_by_title(inp_request)
    end_time = time.time()
    
    print(f"Bi-Encoder 1 search by title took {(end_time-start_time):.3f} seconds")
    print("Top 5 movies with first Bi-Encoder:")
    for movie in recommendations[:5]:
        print(f"\t{movie}")


Please enter a a movie title: Insidious
Bi-Encoder 1 search by title took 2.049 seconds
Top 5 movies with first Bi-Encoder:
	Cherry Tree Lane
	Ghost Voyage
	Quicksilver Highway
	Hostel: Part III
	Darker Than Night


KeyboardInterrupt: ignored